In [0]:
from pyspark import pipelines as dp

In [0]:
@dp.table(
    name = "circuitbox.bronze.addresses_py",
    table_properties = {"quality" : "bronze"}
)
def create_address():
    return(
        spark.readStream.format("cloudFiles") 
            .option("cloudFiles.format", "csv") 
            .option("cloudFiles.inferColumnTypes","true")
            .load("/Volumes/circuitbox/landing/operationaldata/addresses/")
    )

In [0]:
@dp.table(
    name = "circuitbox.silver.addresses_py",
    table_properties = {"quality" : "silver"}
)
@dp.expect_or_fail("validate_customer_id","customer_id is not null")
@dp.expect_or_drop("validate_address","address_line_1 is not null")
def create_address_silver():
    return(
        spark.readStream.table("circuitbox.bronze.addresses_py")
             .drop("_rescued_data")
    )